# Cena com Robô Pioneer 3DX, Móveis e Pessoas: Sistemas de Coordenadas

Este notebook cria uma cena simulada com um robô móvel Pioneer 3DX, três móveis e duas pessoas. Também define sistemas de coordenadas para cada item e mostra algumas transformações entre eles.





In [ ]:
import sys
sys.path.append('./')  # Garante acesso aos módulos locais
import sim  # API remota CoppeliaSim
import numpy as np

def check_return_code(rc):
    assert rc == sim.simx_return_ok, f'Erro na chamada da API: {rc}'

# Conectar ao simulador
sim.simxFinish(-1)
client_id = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
assert client_id != -1, 'Não foi possível conectar ao simulador!'
sim.simxStartSimulation(client_id, sim.simx_opmode_blocking)

In [ ]:
# Função utilitária para obter handle de um objeto
def get_handle(name):
    rc, handle = sim.simxGetObjectHandle(client_id, name, sim.simx_opmode_blocking)
    check_return_code(rc)
    return handle

# Handles dos objetos (ajuste os nomes conforme sua cena)
pioneer = get_handle('Pioneer_p3dx')
movel1 = get_handle('Cuboid')
movel2 = get_handle('Cuboid#0')
movel3 = get_handle('Cuboid#1')
pessoa1 = get_handle('Bill')
pessoa2 = get_handle('Bill#0')

In [ ]:
# Criação de um dummy para cada frame
def criar_dummy(nome, cor=[1,0,0]):
    rc, dummy = sim.simxCreateDummy(client_id, 0.05, None, sim.simx_opmode_blocking)
    check_return_code(rc)
    sim.simxSetObjectIntParameter(client_id, dummy, sim.sim_shapeintparam_respondable, 0, sim.simx_opmode_blocking)
    sim.simxSetObjectIntParameter(client_id, dummy, sim.sim_shapeintparam_static, 1, sim.simx_opmode_blocking)
    sim.simxSetObjectName(client_id, dummy, nome, sim.simx_opmode_blocking)
    sim.simxSetShapeColor(client_id, dummy, None, sim.sim_colorcomponent_ambient_diffuse, cor, sim.simx_opmode_blocking)
    return dummy

# Associando dummies aos objetos
frames = {}
for (name, handle, cor) in [
    ('frame_pioneer', pioneer, [1,0,0]),
    ('frame_movel1', movel1, [0,1,0]),
    ('frame_movel2', movel2, [0,0,1]),
    ('frame_movel3', movel3, [1,1,0]),
    ('frame_pessoa1', pessoa1, [1,0,1]),
    ('frame_pessoa2', pessoa2, [0,1,1]),
]:
    dummy = criar_dummy(name, cor)
    sim.simxSetObjectParent(client_id, dummy, handle, True, sim.simx_opmode_blocking)
    frames[name] = dummy

In [ ]:
# Função para obter posição e orientação absoluta de um frame
def get_pose(handle):
    rc, pos = sim.simxGetObjectPosition(client_id, handle, -1, sim.simx_opmode_blocking)
    check_return_code(rc)
    rc, euler = sim.simxGetObjectOrientation(client_id, handle, -1, sim.simx_opmode_blocking)
    check_return_code(rc)
    return np.array(pos), np.array(euler)

for name, dummy in frames.items():
    pos, euler = get_pose(dummy)
    print(f'{name}: posição = {pos}, orientação (Euler) = {euler}')

In [ ]:
# Exemplo: transformação homogênea do frame do Pioneer para o frame do móvel 1
def get_transform(parent, child):
    rc, matrix = sim.simxGetObjectMatrix(client_id, child, parent, sim.simx_opmode_blocking)
    check_return_code(rc)
    matrix = np.array(matrix).reshape((3,4))
    homog = np.vstack([matrix, [0,0,0,1]])
    return homog

T_pioneer_movel1 = get_transform(frames['frame_pioneer'], frames['frame_movel1'])
print('Transformação homogênea Pioneer->Movel1:')
print(T_pioneer_movel1)

### Encerrar a simluacao no final (nao rodar tudo pra nao rodar essa parte)



In [ ]:
sim.simxStopSimulation(client_id, sim.simx_opmode_blocking)
sim.simxFinish(client_id)

### transformação homogênea

In [ ]:
import numpy as np

def homog_2d(x, y, theta):
    """
    Retorna matriz homogênea 3x3 para pose 2D.
    """
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, x],
                     [s,  c, y],
                     [0,  0, 1]])

### pos global dos objetos

In [ ]:
# Suponha que client_id, handles e nomes estejam definidos (veja células anteriores)
def get_pose_2d(handle):
    rc, pos = sim.simxGetObjectPosition(client_id, handle, -1, sim.simx_opmode_blocking)
    check_return_code(rc)
    rc, euler = sim.simxGetObjectOrientation(client_id, handle, -1, sim.simx_opmode_blocking)
    check_return_code(rc)
    # Considera apenas x, y e theta (rotação em z)
    return np.array([pos[0], pos[1], euler[2]])

poses_global = {}
names_handles = {
    'robô': pioneer,
    'móvel 1': movel1,
    'móvel 2': movel2,
    'móvel 3': movel3,
    'pessoa 1': pessoa1,
    'pessoa 2': pessoa2
}
for nome, handle in names_handles.items():
    poses_global[nome] = get_pose_2d(handle)
    
print(poses_global)

### Matrizes homogêneas de cada elemento no frame do robô

In [ ]:
# Pose global do robô
q = poses_global['robô'] # [x, y, theta]
T_gr = homog_2d(q[0], q[1], q[2])    # Global->Robô
T_rg = np.linalg.inv(T_gr)           # Robô->Global (inversa)

# Para cada elemento, calcule a matriz homogênea no frame do robô
T_robos = {}
for nome, pose in poses_global.items():
    if nome == 'robô':
        continue
    T_gi = homog_2d(pose[0], pose[1], pose[2])   # Global->item
    T_ri = np.dot(T_rg, T_gi)                    # Robô->item = (Robô->Global)^-1 * (Global->item)
    T_robos[nome] = T_ri

for nome, T in T_robos.items():
    print(f'Transformação homogênea {nome} no frame do robô:\n', T)

## Plotar os referenciais e os vetores entre eles

In [ ]:
import matplotlib.pyplot as plt

def plot_frame(T, label, color='k'):
    """
    Plota um referencial 2D (apenas x e y) dado por T (3x3 homogênea).
    """
    origin = T[:2, 2]
    x_axis = origin + T[:2, 0]*0.2
    y_axis = origin + T[:2, 1]*0.2
    plt.plot([origin[0], x_axis[0]], [origin[1], x_axis[1]], color+'-', lw=2)
    plt.plot([origin[0], y_axis[0]], [origin[1], y_axis[1]], color+'--', lw=2)
    plt.text(origin[0], origin[1], label)

# Plote o frame do robô na origem
plt.figure(figsize=(8,8))
plot_frame(np.eye(3), 'robô', 'r')

cores = ['g', 'b', 'm', 'c', 'y']
for i, (nome, T) in enumerate(T_robos.items()):
    plot_frame(T, nome, cores[i % len(cores)])
    # Desenhe um vetor do robô para o objeto
    plt.arrow(0, 0, T[0,2], T[1,2], head_width=0.05, head_length=0.1, fc=cores[i % len(cores)], ec=cores[i % len(cores)], alpha=0.5)

plt.xlabel('x (robô)')
plt.ylabel('y (robô)')
plt.axis('equal')
plt.grid()
plt.title('Referenciais dos objetos no frame do robô')
plt.show()


### exercicio 4: colocar o robo em pos diferentes

In [1]:
# Lista de poses do robô: [x, y, theta]
poses_robo = [
    [0.0, 0.0, 0.0],          # Posição original (de frente)
    [1.0, 0.0, np.pi/2],      # À direita, olhando para cima
    [0.0, -1.0, np.pi],       # Abaixo, olhando para trás
    [-1.0, 1.0, -np.pi/2]     # Esquerda cima, olhando para baixo
]

NameError: name 'np' is not defined

### Função para recalcular e plotar para cada pose

In [ ]:
def plot_cena_para_pose_robo(q, poses_global, nomes_ordenados=None):
    """
    q: [x, y, theta] do robô (no global)
    poses_global: dict nome->[x, y, theta] dos objetos (no global)
    nomes_ordenados: lista de nomes para ordem de cores
    """
    T_gr = homog_2d(q[0], q[1], q[2])  # Global->Robô
    T_rg = np.linalg.inv(T_gr)         # Robô->Global (inversa)
    T_robos = {}
    for nome, pose in poses_global.items():
        if nome == 'robô':
            continue
        T_gi = homog_2d(pose[0], pose[1], pose[2])   # Global->item
        T_ri = np.dot(T_rg, T_gi)                    # Robô->item
        T_robos[nome] = T_ri

    # Plot
    plt.figure(figsize=(8,8))
    plot_frame(np.eye(3), 'robô', 'r')
    cores = ['g', 'b', 'm', 'c', 'y']
    if nomes_ordenados is None:
        nomes_ordenados = list(T_robos.keys())
    for i, nome in enumerate(nomes_ordenados):
        T = T_robos[nome]
        plot_frame(T, nome, cores[i % len(cores)])
        plt.arrow(0, 0, T[0,2], T[1,2], head_width=0.05, head_length=0.1, 
                  fc=cores[i % len(cores)], ec=cores[i % len(cores)], alpha=0.6, length_includes_head=True)

    plt.xlabel('x (robô)')
    plt.ylabel('y (robô)')
    plt.axis('equal')
    plt.grid(True)
    plt.title(f'Referenciais no frame do robô\n(q = [{q[0]:.2f}, {q[1]:.2f}, {np.degrees(q[2]):.1f}°])')
    plt.show()

## plotar novamente

In [ ]:
# (Opcional) Defina a ordem dos nomes (para as cores ficarem consistentes)
nomes_ordenados = ['móvel 1', 'móvel 2', 'móvel 3', 'pessoa 1', 'pessoa 2']

for q in poses_robo:
    plot_cena_para_pose_robo(q, poses_global, nomes_ordenados)